In [1]:
# 네이버 개발자 센터
# 카카오 개발자 센터
# 페이스북 개발자 센터
# 트위터 개발자 센터
# 공공데이터포털
# 세계날씨 http://openweathermap.org
# 유로..... 구글

In [2]:
# 전체작업 설계
# 1. 검색어 지정 : srcText = 000
# 2. 네이버 뉴스 검색 : getNaverSearch()
#     url = base + node + srcText
#     접속및 검색 요청 : urllib.request.urlopen()
#     요청 결과 응답 json으로 받기 : json.load()
# 3. 응답 데이터를 정리해서 리스트로 저장 : getPostData()
# 4. 리스트를 JSON 파일로 저장 : json.dumps()

In [3]:
# 시퀀스
# 검색어 지정 -> 뉴스검색->getNaverSearch()         ->         getRequestUrl()
#                   <- json.load(responseDecode) <- Response.read()
# 리스트에 저장 --------->   getPostData()
#                <------ jsonResult

In [7]:
import urllib
import datetime
import json

In [9]:
def getPostData(post,jsonResult,cnt):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    pDate = datetime.datetime.strptime(post['pubDate'],'%a, %d %b %Y %H:%M:%S +0900' )
    pDate = pDate.strptime('%Y-%m-%d %H:%M:%S')
    jsonResult.append(
        {
                'cnt':cnt, 'title':title, 'description':description
                ,'org_link':org_link, 'link':link, 'pDate':pDate
        })
    return

In [10]:
def getNaverSearch(node,srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/"+node+".json"
    parameters = f"?query={urllib.parse.quote(srcText)}&start={start}&display={display}"
    url = base+node+parameters
    responseDecode = getRequestUrl(url)
    if(responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [11]:
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id",'1thcHFTKX2Brlcp2Y6PA')
    req.add_header("X-Naver-Client-Secret",'syAhbzt28N')
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print(f"{datetime.datetime.now} url request success")
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print(f"{datetime.datetime.now} error for url : {url}")
    

In [12]:
# 함수설계하기
def main():
    node = 'news' # 크롤링할 대상
    srcText = input('검색어를 입력하세요')
    cnt = 0
    jsonResult = []
    jsonResponse = getNaverSearch(node,srcText,1,100) # start=1, display=100
    total = jsonResponse['total']
    while( (jsonResponse!=None) and (jsonResponse['display']!=0) ):
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post,jsonResult,cnt)
        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node,srcText,start,100)
    print(f'전체검색: {total}건')
    print(f'가져온 데이터: {cnt}건')         
        